In [1]:


import matplotlib.pyplot as plt

from chord import Chord
# load 
import sys
import time

from sklearn.model_selection import cross_val_score,train_test_split
from datetime import date
import numpy as np
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import pingouin as pg
import subprocess
import os 
from scipy.stats import ranksums
from scipy.stats import pearsonr
from scipy import stats
import seaborn as sns
from scipy.stats import mannwhitneyu
if sys.platform == 'darwin':
    print("Current system is macOS")
    main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
elif sys.platform == 'linux':
    print("Current system is Ubuntu")
    main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
else:
    print("Current system is neither macOS nor Ubuntu")
    
    
os.chdir(main_fold_path)


from utils.fnirs_utils import get_nine_region_data


    
DATA =  np.load('allData/prognosis_mix_hb/pretreatment_response/hb_data.npy')
LABEL =  np.load('allData/prognosis_mix_hb/pretreatment_response/label.npy')
data = DATA
label = LABEL
data = get_nine_region_data(data)

x = np.load('allData/prognosis_mix_hb/pretreatment_response/label_hamd.npy')
reduced_percentage = (x[:, 1] - x[:, 0]) / x[:, 0]

HbO = np.transpose(data[...,0::2],(0,2,1))
# HbO = individual_normalization(HbO)
print(f'HbO: {HbO.shape}')
HbR = np.transpose(data[...,1::2],(0,2,1))
HbT = HbO + HbR

def set_xlabel_of_ax(ax, label_name, font_size):
    if label_name == 'HAMD_change':
        ax.set_xlabel('HAMD change', fontsize=font_size)
        ax.set_xticks(np.arange(-1,1, 0.5))
        
    else:
        ax.set_xticks([0, 1])
        ax.set_xticklabels(['Nonresponders', 'Responders'], fontsize=font_size)
            
def get_five_regions_from_nine(data):
    res = np.zeros((data.shape[0], 5))
    res[:, 0] = (data[:, 0] + data[:, 1]) / 2
    res[:, 1] = (data[:, 2] + data[:, 3]) / 2
    res[:, 2] = (data[:, 4] + data[:, 5]) / 2
    res[:, 3] = (data[:, 6] + data[:, 7]) / 2
    res[:, 4] = data[:, 8]
    return res 

task_end_index = 700
task_start_index = 100
nine_region_name = ['L-pSFC', 'R-pSFC', 'L-dlPFC', 'R-dlPFC', 'L-STG', 'R-STG', 'L-vlPFC', 'R-vlPFC', 'mPFC']

five_region_name = ['pSFC', 'dlPFC', 'STG', 'vlPFC', 'mPFC']

dict_hb = {'HbO': HbO, 'HbR': HbR, 'HbT': HbT}

dict_label = {'HAMD_change': reduced_percentage, 'Responders': label}

dict_region = {'five_regions': five_region_name, 'nine_regions': nine_region_name}

feature_name_all = ['Task change', 'Mean', 'Task activation']

HB = HbT
hb_type = 'HbT'

region_num = 'nine_regions'

output_fold = 'FigureTable/regression'

res_of_dpc = []
res_of_dpc_name = []

res_of_pSFC = []
res_of_pSFC_name = []
for feature_name in feature_name_all:
    for region_num, region_name in dict_region.items():
        for label_name, label_val in dict_label.items():
            for hb_type, HB in dict_hb.items():
                task_change_hb = np.zeros((64, 1, 9))
                
                for i in range(task_change_hb.shape[-1]):
                    if feature_name == 'Task change': x = HB[:, task_end_index:, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1)
                    if feature_name == 'Mean': x = HB[:, :, i].mean(axis=1) 
                    if feature_name == 'Task activation': x = HB[:, task_start_index:task_end_index, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1) - HB[:, task_end_index:, i].mean(axis=1)
                    

Current system is Ubuntu
9
[array([ 8,  9, 19, 20]), array([ 0,  1, 10, 11]), array([ 6,  7, 16, 17, 18, 27, 28]), array([ 2,  3, 12, 13, 14, 23, 24]), array([29, 30, 40, 41, 50, 51]), array([21, 22, 31, 32, 42, 43]), array([38, 39, 48, 49]), array([33, 34, 44, 45]), array([ 4,  5, 15, 25, 26, 35, 36, 37, 46, 47])]
HbO: (64, 1250, 9)


In [15]:
def calcualte_fnirs_feature(hb, start_index, end_index, feature_name):
    res = None
    if feature_name == 'Task change': res = hb[:, end_index:, :].mean(axis=1) - hb[:, :start_index, :].mean(axis=1)
    if feature_name == 'Mean': res =  hb[:, :, :].mean(axis=1)
    if feature_name == 'Task activation': res =  hb[:, start_index:end_index, :].mean(axis=1) - hb[:, :start_index, :].mean(axis=1) - hb[:, end_index:, :].mean(axis=1)
    
    if res is not None:
        return res
    else:
        raise ValueError('feature_name is not valid')
    

In [44]:
def get_five_regions_from_nine(data):
    res = np.zeros((data.shape[0], 5, data.shape[-1]))
    res[:, 0, :] = (data[:, 0, :] + data[:, 1, :]) / 2
    res[:, 1, :] = (data[:, 2, :] + data[:, 3, :]) / 2
    res[:, 2, :] = (data[:, 4, :] + data[:, 5, :]) / 2
    res[:, 3, :] = (data[:, 6, :] + data[:, 7, :]) / 2
    res[:, 4, :] = data[:, 8, :]
    return res 

def compute_fNIRS_feature_save(HbO, HbR, HbT, task_start_index, task_end_index, feature_name, region_name, save_pth):
    feature_hb = []
    for hb in [HbO, HbR, HbT]:
        hb_feature = calcualte_fnirs_feature(hb, task_start_index, task_end_index, feature_name)
        feature_hb.append(hb_feature)
    feature_hb = np.array(feature_hb)
    feature_hb = np.transpose(feature_hb, (1, 2, 0))
    if region_name == 'five_regions':
        feature_hb = get_five_regions_from_nine(feature_hb)
    print(feature_name, feature_hb.shape,' is saved to ', save_pth)
    np.save(save_pth, feature_hb)
    

In [25]:
task_end_index = 700
task_start_index = 100

fnirs_feature_name = ['Task change', 'Mean']
regions_name = ['nine_regions', 'five_regions']
save_pth = 'allData/prognosis_mix_hb/pretreatment_response/task_change_hb_nine_regions.npy'
compute_fNIRS_feature_save(HbO, HbR, HbT, task_start_index, task_end_index, 'Task change', save_pth)



In [45]:
fnirs_feature_name = ['Task change', 'Mean']
regions_name = ['nine_regions', 'five_regions']
for i in fnirs_feature_name:
    for region_name in regions_name:
        feature_name = i.replace(' ', '_').lower()
        save_pth = 'allData/prognosis_mix_hb/pretreatment_response/' + feature_name + '_hb_' + region_name + '.npy'
        compute_fNIRS_feature_save(HbO, HbR, HbT, task_start_index, task_end_index, i, region_name,save_pth)


Task change (64, 9, 3)  is saved to  allData/prognosis_mix_hb/pretreatment_response/task_change_hb_nine_regions.npy
Task change (64, 5, 3)  is saved to  allData/prognosis_mix_hb/pretreatment_response/task_change_hb_five_regions.npy
Mean (64, 9, 3)  is saved to  allData/prognosis_mix_hb/pretreatment_response/mean_hb_nine_regions.npy
Mean (64, 5, 3)  is saved to  allData/prognosis_mix_hb/pretreatment_response/mean_hb_five_regions.npy


: 

In [21]:
task_end_index = 700
task_start_index = 100
save_pth = 'allData/prognosis_mix_hb/pretreatment_response/mean_hb_nine_regions.npy'


feature_hb = []
for hb in [HbO, HbR, HbT]:
    hb_feature = calcualte_fnirs_feature(hb, task_start_index, task_end_index, 'Mean')
    feature_hb.append(hb_feature)
feature_hb = np.array(feature_hb)
feature_hb = np.transpose(feature_hb, (1, 2, 0))
np.save(save_pth, feature_hb)
print(np.mean(feature_hb))

0.3102146091270153
